#Leitura dos dados

In [1]:
!pip install pyspark==3.3.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 41 kB/s 
     |████████████████████████████████| 199 kB 66.6 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=3edc4d234cbd5086cefbeff7bd21d60f02e6b3b9ee210f58ecdf8f92775b68b5
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark


In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.master('local[*]').appName("Transformacao com Spark").getOrCreate()

spark

In [4]:
!wget "https://caelum-online-public.s3.amazonaws.com/challenge-spark/semana-2.zip" && unzip semana-2.zip -d dados/

--2022-12-02 12:26:34--  https://caelum-online-public.s3.amazonaws.com/challenge-spark/semana-2.zip
Resolving caelum-online-public.s3.amazonaws.com (caelum-online-public.s3.amazonaws.com)... 52.217.231.1, 54.231.139.145, 52.216.62.153, ...
Connecting to caelum-online-public.s3.amazonaws.com (caelum-online-public.s3.amazonaws.com)|52.217.231.1|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2763652 (2.6M) [application/zip]
Saving to: ‘semana-2.zip’

semana-2.zip        100%[===================>]   2.63M  --.-KB/s    in 0.1s    

2022-12-02 12:26:35 (19.2 MB/s) - ‘semana-2.zip’ saved [2763652/2763652]

Archive:  semana-2.zip
   creating: dados/dataset_transformado_parquet/
  inflating: dados/dataset_transformado_parquet/_SUCCESS  
  inflating: dados/dataset_transformado_parquet/._SUCCESS.crc  
  inflating: dados/dataset_transformado_parquet/part-00000-00341ba7-0a7c-4fef-a81e-1066725a64b1-c000.snappy.parquet  
  inflating: dados/dataset_transformado_parquet/.part

In [5]:
dados = spark.read.parquet('/content/dados/dataset_transformado_parquet')

In [6]:
dados.show(truncate=False)

+------------------------------------+-----+----------+---------+---------+-----------------------------------------------------------------------------------------------------------------------------------------+-------+------+------------+------------+-----------+----+-----------------------+------------+----------+----+-----+------+
|id                                  |andar|area_total|area_util|banheiros|caracteristicas                                                                                                                          |quartos|suites|tipo_anuncio|tipo_unidade|tipo_uso   |vaga|bairro                 |zona        |condominio|iptu|tipo |valor |
+------------------------------------+-----+----------+---------+---------+-----------------------------------------------------------------------------------------------------------------------------------------+-------+------+------------+------------+-----------+----+-----------------------+------------+----------+----+

In [7]:
dados.printSchema()

root
 |-- id: string (nullable = true)
 |-- andar: long (nullable = true)
 |-- area_total: string (nullable = true)
 |-- area_util: string (nullable = true)
 |-- banheiros: long (nullable = true)
 |-- caracteristicas: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- quartos: long (nullable = true)
 |-- suites: long (nullable = true)
 |-- tipo_anuncio: string (nullable = true)
 |-- tipo_unidade: string (nullable = true)
 |-- tipo_uso: string (nullable = true)
 |-- vaga: long (nullable = true)
 |-- bairro: string (nullable = true)
 |-- zona: string (nullable = true)
 |-- condominio: string (nullable = true)
 |-- iptu: string (nullable = true)
 |-- tipo: string (nullable = true)
 |-- valor: string (nullable = true)



#Analisando a diferença entre a quantidade da dados não nulos armazenados nas colunas 'area_util' e 'area_total'

In [8]:
from pyspark.sql import functions as f

In [9]:
dados.select('area_total').where(f.col('area_total').isNotNull()).count()

57368

In [10]:
dados.select('area_util').where(f.col('area_util').isNotNull()).count()

66562

In [11]:
dados = dados.drop('area_total')

In [12]:
dados.printSchema()

root
 |-- id: string (nullable = true)
 |-- andar: long (nullable = true)
 |-- area_util: string (nullable = true)
 |-- banheiros: long (nullable = true)
 |-- caracteristicas: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- quartos: long (nullable = true)
 |-- suites: long (nullable = true)
 |-- tipo_anuncio: string (nullable = true)
 |-- tipo_unidade: string (nullable = true)
 |-- tipo_uso: string (nullable = true)
 |-- vaga: long (nullable = true)
 |-- bairro: string (nullable = true)
 |-- zona: string (nullable = true)
 |-- condominio: string (nullable = true)
 |-- iptu: string (nullable = true)
 |-- tipo: string (nullable = true)
 |-- valor: string (nullable = true)



#Convertendo os tipos das colunas

In [13]:
dados = dados.withColumn('area_util', dados.area_util.cast('double'))
dados = dados.withColumn('iptu', dados.iptu.cast('double'))
dados = dados.withColumn('valor', dados.valor.cast('double'))
dados = dados.withColumn('condominio', dados.condominio.cast('double'))

dados = dados.withColumn('andar', dados.andar.cast('integer'))
dados = dados.withColumn('banheiros', dados.banheiros.cast('integer'))
dados = dados.withColumn('quartos', dados.quartos.cast('integer'))
dados = dados.withColumn('suites', dados.suites.cast('integer'))

In [14]:
dados.printSchema()

root
 |-- id: string (nullable = true)
 |-- andar: integer (nullable = true)
 |-- area_util: double (nullable = true)
 |-- banheiros: integer (nullable = true)
 |-- caracteristicas: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- quartos: integer (nullable = true)
 |-- suites: integer (nullable = true)
 |-- tipo_anuncio: string (nullable = true)
 |-- tipo_unidade: string (nullable = true)
 |-- tipo_uso: string (nullable = true)
 |-- vaga: long (nullable = true)
 |-- bairro: string (nullable = true)
 |-- zona: string (nullable = true)
 |-- condominio: double (nullable = true)
 |-- iptu: double (nullable = true)
 |-- tipo: string (nullable = true)
 |-- valor: double (nullable = true)



#Tratamento da coluna caracteristicas

In [19]:
caracteristicas = dados.select(dados.id,f.explode(dados.caracteristicas))
caracteristicas.show()
caracteristicas.groupBy('col').count().show()

+--------------------+------------------+
|                  id|               col|
+--------------------+------------------+
|03a386b6-7ab8-4ef...|     Churrasqueira|
|03a386b6-7ab8-4ef...|          Academia|
|03a386b6-7ab8-4ef...|        Playground|
|03a386b6-7ab8-4ef...|   Salão de festas|
|03a386b6-7ab8-4ef...|Condomínio fechado|
|03a386b6-7ab8-4ef...| Portão eletrônico|
|03a386b6-7ab8-4ef...|      Portaria 24h|
|03a386b6-7ab8-4ef...|Animais permitidos|
|40a2cbde-946b-42d...|Condomínio fechado|
|40a2cbde-946b-42d...|      Portaria 24h|
|40a2cbde-946b-42d...|   Salão de festas|
|40a2cbde-946b-42d...|     Churrasqueira|
|40a2cbde-946b-42d...|           Piscina|
|40a2cbde-946b-42d...|Animais permitidos|
|be269ddf-a4a3-4a7...|     Churrasqueira|
|be269ddf-a4a3-4a7...|Condomínio fechado|
|be269ddf-a4a3-4a7...|           Piscina|
|4b7e3d59-34fd-4ba...|     Churrasqueira|
|4b7e3d59-34fd-4ba...|           Piscina|
|4b7e3d59-34fd-4ba...|        Playground|
+--------------------+------------

In [24]:
caracteristicas = caracteristicas.groupBy('id').pivot('col').agg(f.lit(1)).na.fill(0)
caracteristicas.show()

+--------------------+--------+------------------+-------------+------------------+--------+-------+----------+------------+-----------------+---------------+
|                  id|Academia|Animais permitidos|Churrasqueira|Condomínio fechado|Elevador|Piscina|Playground|Portaria 24h|Portão eletrônico|Salão de festas|
+--------------------+--------+------------------+-------------+------------------+--------+-------+----------+------------+-----------------+---------------+
|fd96bbd5-d631-416...|       1|                 1|            1|                 1|       1|      1|         1|           1|                1|              1|
|bfffedfe-99e7-4ae...|       0|                 1|            1|                 1|       1|      0|         1|           0|                1|              1|
|fcb67af3-5601-415...|       1|                 1|            1|                 0|       0|      1|         1|           0|                0|              1|
|afecddff-f4cc-4ab...|       1|               

In [27]:
dados = dados.join(caracteristicas, 'id', how='inner').drop('caracteristicas')
dados.show()

+--------------------+-----+---------+---------+-------+------+------------+------------+-----------+----+--------------------+------------+----------+-------+-----+---------+--------+------------------+-------------+------------------+--------+-------+----------+------------+-----------------+---------------+
|                  id|andar|area_util|banheiros|quartos|suites|tipo_anuncio|tipo_unidade|   tipo_uso|vaga|              bairro|        zona|condominio|   iptu| tipo|    valor|Academia|Animais permitidos|Churrasqueira|Condomínio fechado|Elevador|Piscina|Playground|Portaria 24h|Portão eletrônico|Salão de festas|
+--------------------+-----+---------+---------+-------+------+------------+------------+-----------+----+--------------------+------------+----------+-------+-----+---------+--------+------------------+-------------+------------------+--------+-------+----------+------------+-----------------+---------------+
|fd96bbd5-d631-416...|    0|     48.0|        1|      2|     1| 